In [2]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

import pendulum
import tqdm

In [4]:
# assume we set theta and dtheta = 0 and u = -5, we can get the next state using
x = np.array([0,0])
u = -5
x_next = pendulum.get_next_state(x, u)

In [5]:
discretized_theta = np.linspace(0, 2*np.pi, 50, endpoint=False)

discretized_thetadot = np.linspace(-6, 6, 50)
def get_cost(x,u):
    theta = x[0]
    theta_dot = x[1]
    res = (theta-np.pi)**2+0.01*theta_dot**2+0.0001*u**2
    return res

In [ ]:
class Q_solver:
    def __init__(self, costfn = get_cost, actions=None, max_iters=5000, sparse_loss=False):
        if actions is None:
            actions = [-5, 0, 5]
        self.action_list = np.array(actions)
        self.lr = 0.1
        self.epslion = 0.1
        self.step_in_iter = 100
        self.alpha = 0.99
        self.max_iters = max_iters
        self.sparse_loss = sparse_loss
        self.costfn = costfn
        self.discretized_theta = discretized_theta
        self.discretized_theta_dot = discretized_thetadot
        self.space_shape = [len(discretized_theta),len(discretized_thetadot)]

        self.num_states = 50*50
        self.nu = 3
        self.nq = 50
        self.make_state_transfer_table()
    def make_state_transfer_table(self):
        next_state_index = np.empty([self.num_states, self.nu], dtype=np.int32)
        for i in range(self.num_states):
            for k in range(self.nu):
                x_next = pendulum.get_next_state(self.get_states(i), self.action_list[k])
                next_state_index[i, k] = self.get_index(x_next)

        self.state_transfer_table = next_state_index #[250 3 2]


    def get_index(self,x):
        ind_q = np.argmin((x[0] - self.discretized_theta) ** 2)
        ind_v = np.argmin((x[1] - self.discretized_theta_dot) ** 2)
        return ind_q + ind_v * self.nq

    def get_states(self, index):
        iv, ix = np.divmod(index, self.nq)
        return np.array([self.discretized_theta[ix], self.discretized_theta_dot[iv]])


    def get_policy_and_value_function(self):
        q = np.zeros([self.num_states, self.nu])
        q_Last = np.zeros([self.num_states, self.nu])
        for i in tqdm.tqdm(range(self.max_iters)):  #

            # choose initial state x0
            x_0 = np.array([0, 0])
            x_index = self.get_index(x_0)
            for j in range(self.step_in_iter):

                if np.random.uniform(0, 1) > self.epslion:
                    u_index = np.argmin(q[x_index, :])
                else:
                    u_index = np.random.randint(0, self.nu - 1)
                # observe x_t+1
                next_index = self.state_transfer_table[x_index, u_index]
                # compute g(x_t,u(x_t))
                x = self.get_states(x_index)
                u = self.action_list[u_index]
                # compute TDerror
                TDerror = self.costfn(x, u) + self.alpha * min(q[next_index, :]) - q[
                    x_index, u_index]
                q[x_index, u_index] = q[x_index, u_index] + self.lr * TDerror
                x_index = next_index

            # we update the current Q function if there is any change otherwise we are done
            if ((q_Last - q) ** 2 < 1e-5).all():
                break
            else:
                q_Last = q.copy()

        policy = np.zeros(self.space_shape)
        value_function = np.zeros(self.space_shape)
        for k in range(self.num_states):
            iv, ix = np.divmod(k, self.nq)
            policy[ix,iv] = self.action_list[np.argmin(q[k, :])]
            value_function[ix,iv] = min(q[k, :])
        return value_function,policy
solver = Q_solver(get_cost,max_iters=50000)
value,policy = solver.get_policy_and_value_function()

In [ ]:
policy

In [ ]:
# here is some code to plot results, assuming a policy and a value function are given
# this can be used to answer questions in both Part 1 and 2

# we plot the value function
plt.figure(figsize=[6,6])
plt.imshow(value.T, extent=[0., 2*np.pi, -6, 6], aspect='auto')
plt.xlabel('Pendulum Angle')
plt.ylabel('Velocity')
plt.title('Value Function')

# we plot the policy
plt.figure(figsize=[6,6])
plt.imshow(policy.T, extent=[0., 2*np.pi, -6, 6], aspect='auto')
plt.xlabel('Pendulum Angle')
plt.ylabel('Velocity')
plt.title('Policy')

# now we simulate the dynamics for 100 time steps
x0 = np.array([0.,0.])

def controller(x):
    theta = np.linspace(0, 2*np.pi, 50, endpoint=False)
    dtheta = np.linspace(-6, 6, 50)
    
    th_index = np.argmin(np.abs(theta - x[0]))
    dth_index = np.argmin(np.abs(dtheta - x[1]))
    return policy[th_index, dth_index]



In [ ]:
t, x, u = pendulum.simulate(x0, controller, 30)

# and plot the results
time = np.linspace(0.,20., len(x[0,:]))
plt.figure()
plt.subplot(3,1,1)
plt.plot(time,x[0,:])
plt.ylabel('angle')
plt.subplot(3,1,2)
plt.plot(time,x[1,:])
plt.ylabel('velocity')
plt.subplot(3,1,3)
plt.plot(time[:-1],u)
plt.ylabel('control')

In [ ]:
pendulum.animate_robot(x)